In [56]:
import pandas as pd
import numpy as np

In [ ]:
!pip install colorama
!pip install pydantic-settings
!pip install pandas-profiling

In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
from PIL import Image as im
from wordcloud import WordCloud,STOPWORDS
from IPython.core.display import Image
from colorama import Fore, Back, Style
from pydantic_settings import BaseSettings
from pandas_profiling import ProfileReport
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

<ipython-input-5-f93c2064a9ef>:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [ ]:
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import requests
custom_colors = ['#48bfe3','#56cfe1','#64dfdf','#72efdd','#80ffdb']
customPalette = sns.color_palette(custom_colors)
#sns.palplot(sns.color_palette(custom_colors),size=1)

sns.set_context("poster")

In [ ]:
raw = pd.read_csv("/content/train.csv")

In [ ]:
 profile = pandas_profiling.ProfileReport(df)
 profile.to_file(output_file = 'EDA.html' )

# 전처리 부분

## cleansing

In [71]:
from sklearn.preprocessing import LabelEncoder
import re
pattern = r'[^\w\s\d]'

def FE(x):
    # 저자, 출판사 특수문자, 공백 제거
    x['Book-Author'] = x['Book-Author'].astype(str)
    x['Publisher'] = x['Publisher'].astype(str)
    x['Book-Author'] = x['Book-Author'].apply(lambda x: re.sub(pattern, '', x))
    x['Publisher'] = x['Publisher'].apply(lambda x: re.sub(pattern, '', x))
    x['Book-Author'] = x['Book-Author'].replace('\s+', ' ', regex=True)
    x['Publisher'] = x['Publisher'].replace('\s+', ' ', regex=True)
    x['Book-Author'] = x['Book-Author'].str.replace(' ', '')
    x['Publisher'] = x['Publisher'].str.replace(' ', '')
    x['Book-Author'] = x['Book-Author'].str.lower()
    x['Publisher'] = x['Publisher'].str.lower()

    # 책 타이틀도 공백제거, 문자열만 남기기.
    x['Book-Title'] = x['Book-Title'].apply(lambda x: re.sub(pattern, '', x))
    x['Book-Title'] = x['Book-Title'].replace('\s+', ' ', regex=True)
    x['Book-Title'] = x['Book-Title'].apply(lambda x:x.strip())
    x['Book-Title'] = x['Book-Title'].str.lower()

    x = x.replace({'no': 'n/a'})
    return x

df = FE(raw)


### 작가 처리

In [72]:
# 같은 작가나 작가 명 다른 경우 같은 작가로 친다
def find_author(author):
    if author in ['joannekrowling', 'rowlingjk']:
        return 'jkrowling'
    if author == 'shakespeare':
        return 'williamshakespeare'
    return author

df['Book-Author'] = df['Book-Author'].apply(find_author)


### 나라 처리

In [95]:
def find_country(country_list:str):
    if country_list in ['galiza']:
        return 'spain'
    elif country_list in ['united sates','maricopa','richmond country','framingham','usa (currently living in england)','saint loius','shelby',
                     'los estados unidos de norte america','united states','polk','u.s.a.','morgan','us','ventura county','hernando','kern',
                     'onondaga nation','disgruntled states of america','san mateo','united state','aroostook','guam','fort bend',
                      'hungary and usa','u.s. of a.','united staes','america','alachua','u.s>']:
        return 'usa'
    elif country_list in ['the world tomorrow', 'galiza neghra','universe','Invalid space count','n/a','tdzimi','space','here and there',
                     'cherokee','csa','x','we`re global!','unknown','nz','ysa','n/a - on the road','the great white north','far away...',
                      'quit','k1c7b1','caribbean sea','st.thomasi','rutherford','"n/a"','the','st. helena','c',
                      'everywhere and anywhere']:
        return 'other'
    elif country_list in ['jersey','la france']:
        return ' france'
    elif country_list in ['catalunya','lleida','catalonia','berguedà','catalunya spain','madrid','orense']:
        return ' spain'
    elif country_list in [' la chine éternelle !',' macau']:
        return ' china'
    elif country_list in ['alderney','worcester','guernsey','prince william','bermuda','cayman islands','channel islands','u.k.',
                     'camden']:
        return 'united kingdom'
    elif country_list in ['collin']:
        return 'czech republic'
    elif country_list in ['burma']:
        return 'myanmar'
    elif country_list in ['united arab emirates']:
        return 'u.a.e'
    elif country_list in ['pender']:
        return 'poland'
    elif country_list in ['tobago']:
        return 'trinidad and tobago'
    elif country_list in ['urugua']:
        return 'uruguay'
    elif country_list in ['ferrara','italia','l`italia']:
        return 'italy'
    elif country_list in ['niger']:
        return 'nigeria'
    elif country_list in ['la belgique']:
        print(1)
        return 'belgium'
    elif country_list in ['burlington','cananda']:
        return 'canada'
    return country_list



In [88]:
df['location_city'] = df['Location'].apply(lambda x: x.split(',')[0].strip())
df['location_state'] = df['Location'].apply(lambda x: x.split(',')[1].strip())
df['location_country'] = df['Location'].apply(lambda x: x.split(',')[2].strip())
df = df.replace('na', np.nan)
df = df.replace('', np.nan)


In [89]:
df.drop(columns=['Location'], inplace=True)

In [96]:
df['location_country']=df['location_country'].apply(lambda x: find_country(x))

## 나이 처리

In [103]:
bins = [-1, 9, 19, 29, 39, 49, 59, 69, 79, np.inf]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
df['Age'] = pd.cut(df['Age'], bins=bins, labels=labels)

# Age 컬럼에 대한 레이블 인코딩
age_le = LabelEncoder()
df['Age'] = age_le.fit_transform(df['Age'])

### 출판연도 처리

In [111]:
bins = [-1, 1969, 1979, 1989, 1999,  np.inf]
labels = ['1960', '1970', '1980', '1990', '2000+']
df['Year-Of-Publication'] = pd.cut(df['Year-Of-Publication'], bins=bins, labels=labels)

# Age 컬럼에 대한 레이블 인코딩
age_le = LabelEncoder()
df['Year-Of-Publication'] = age_le.fit_transform(df['Year-Of-Publication'])

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   871393 non-null  object
 1   User-ID              871393 non-null  object
 2   Book-ID              871393 non-null  object
 3   Book-Rating          871393 non-null  int8  
 4   Age                  871393 non-null  int64 
 5   Book-Title           871393 non-null  object
 6   Book-Author          871386 non-null  object
 7   Year-Of-Publication  871393 non-null  int64 
 8   Publisher            871393 non-null  object
 9   location_city        871078 non-null  object
 10  location_state       852630 non-null  object
 11  location_country     850257 non-null  object
dtypes: int64(2), int8(1), object(9)
memory usage: 74.0+ MB


### Book Rating 처리

In [114]:
df.to_csv('fixed_book_data.csv')